In [3]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

# 로그 추적을 위해서는 LangSmith를 통해 가입하여 환경 변수를 설정해야 합니다.
# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true


In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# 무료 Open Source 기반 임베딩
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [6]:
import os
print(os.getcwd())

/Users/songseung-yun/Desktop/RAG


### 1단계: 문서 로드 ( 도서 대출 내역 csv파일 + 한성대학교 이용 세칙 html 웹 로드)

In [7]:
# 단계 1: 문서 로드(Load Documents)
# 한성대학교 학술 정보관 이용에 관한 세칙을 로드하고 청크로 나누고 인덱싱 합니다.

# 학술정보관 세칙에 관련한 web문서 업로드
url = "https://hsel.hansung.ac.kr/intro_data.mir"
loader = WebBaseLoader(
    web_path=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"id": "intro_rule"}
              ),
    )
)
url_docs = loader.load()
print(url_docs)

[Document(metadata={'source': 'https://hsel.hansung.ac.kr/intro_data.mir'}, page_content='\n\n학술정보관 이용에 관한 세칙\n 제 1 조 (목적)\n\n본 세칙은 한성대학교 학술정보관규정 제5장 자료의 이용을 보완하고, 위임된 세부사항을 정함으로써 관련 업무 처리의 명확한 기준을 마련하고 효율적으로 지원함에 그 목적이 있다.\n\n 제 2 조 (개관 및 열람 시간)\n\n도서관의 개관 및 열람 시간은 다음과 같다. 다만, 관장이 필요하다고 인정한 때에는 개관시간을 변경할 수 있다.\n\r\n\t\t\t\t\t\t\t\t\t\t\t1. 자료열람실 : 학기 중 평일 09:00～21:00 토요일 11:00 ~ 15:00 학기 중 방학 중 평일 10:00～16:00 토요일 휴관\r\n\t\t\t\t\t\t\t\t\t\t\t2. 일반열람실 : 연중 06:30～23:00\r\n\t\t\t\t\t\t\t\t\t\t\n\n\n 제 3 조 (휴관일)\n\n\r\n\t\t\t\t\t\t\t\t\t\t① 도서관의 휴관일은 다음과 같다. 휴관 시설은 자료열람실에 한한다.\n\r\n\t\t\t\t\t\t\t\t\t\t\t1. 일요일\r\n\t\t\t\t\t\t\t\t\t\t\t2. 법정공휴일\r\n\t\t\t\t\t\t\t\t\t\t\t3. 개교기념일\r\n\t\t\t\t\t\t\t\t\t\t\n\n② ①항에 포함되지 않더라도 관장이 필요성을 인정한 때에는 자료실 및 열람실을 임시로 휴관 또는 개관할 수 있다.\n\n 제 4 조 (그 밖에 관장의 허가를 얻은 자의 이용 자격)\n\n\r\n\t\t\t\t\t\t\t\t\t\t① 학술정보관규정 제23조(열람대상) 제5호 그 밖에 관장이 승인한 사람은 다음 각 호와 같다. \n\r\n\t\t\t\t\t\t\t\t\t\t\t1. 비전임 교원 및 강사\r\n\t\t\t\t\t\t\t\t\t\t\t2. 본 대학교와 대학원의 휴학 및 수료생\r\n\t\t\t\t\t\t\t

In [8]:
from langchain_community.document_loaders import WebBaseLoader, TextLoader, CSVLoader

# 도서 대출 내역 csv 파일 로드
# 대출월 기반 
# CSV 파일
csv_loader = CSVLoader(file_path="book_data_RAG_location.csv", encoding="utf-8")
csv_docs = csv_loader.load()

# 문서 합치기
docs = csv_docs + url_docs

In [9]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, # 한 번에 분할되는 텍스트(청크)의 최대 길이
    chunk_overlap=50, # 청크 간 중복 길이
    length_function=len, # 텍스트 길이 계산 함수
    is_separator_regex=False, # 텍스트 분할 시 정규식 사용 여부
)

splits = text_splitter.split_documents(docs)
print(splits[:3])  # 처음 3개만 출력
print(len(splits))  # 전체 개수만 출력

[Document(metadata={'source': 'book_data_RAG_location.csv', 'row': 0}, page_content='서명: 세잔느의 眞實\n저자: 미셸 후그 著\n출판사: 열화당\n출판년도: 1988\n청구기호: 650.1 ㅎ891ㅅ\n대출일자: 2015-09-21\n신분: 교직원\n소속(학과): 컴퓨터공학부\n학번: 교직원\n학년: \n대출횟수: 1\n보존서가 소장처: Design&amp;IT정보센터(6F)\n보존서가 칸: 8-A 5-e'), Document(metadata={'source': 'book_data_RAG_location.csv', 'row': 1}, page_content='서명: 여성이 갖고 있는 남성의 이미지\n저자: 사라 켄트\n출판사: 삼신각\n출판년도: 1996\n청구기호: 600.453 ㅋ428ㅇ\n대출일자: 2021-03-11\n신분: 교직원\n소속(학과): 컴퓨터공학부\n학번: 교직원\n학년: \n대출횟수: 1\n보존서가 소장처: Design&amp;IT정보센터(6F)\n보존서가 칸: 2-A 7-e'), Document(metadata={'source': 'book_data_RAG_location.csv', 'row': 2}, page_content='서명: 서양미술사\n저자: E. H. 곰브리치 지음\n출판사: 예경,\n출판년도: 1997\n청구기호: 609.2 ㄱ428ㅅ\n대출일자: 2022-06-22\n신분: 교직원\n소속(학과): 컴퓨터공학부\n학번: 교직원\n학년: \n대출횟수: 2\n보존서가 소장처: Design&amp;IT정보센터(6F)\n보존서가 칸: 3-B 8-a')]
26437


# 3단계 + 4단계: 임베딩, 벡터스토어를 생성한다.
- 임베딩이란? 텍스트, 이미지, 오디오 등 다양한 데이터를 벡터 형태로 표현하는 방법입니다. 이렇게 변환된 벡터는 데이터 간의 관계를 파악하고 유사성을 분석하는 데 활용됩니다
- 임베딩을 마친후 벡터들을 담을 벡터스토어를 생성한다.
- 위에서 텍스트를 청크로 분할한 splits를 기준으로 벡터스토어를 생성하여 내부에 담아낸다.

In [ ]:
# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# HuggingFace 오픈소스 무료 임베딩으로 벡터스토어를 생성합니다.
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

In [10]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [11]:
# openAI 임베딩 벡터스토어
# openAIvectorstore = FAISS.from_documents(
#     documents=splits, embedding=OpenAIEmbeddings()
# )

## 5단계: Retriever 생성
- 리트리버는 구조화되지 않은 쿼리가 주어지면 문서를 반환하는 인터페이스이다.
- 리트리버는 문서를 저장할 필요 없이 문서를 반환(또는 검색)하기만 합니다.
- 생성된 VectorStore에 as_retreiver()로 가져와서 Retriever를 생성합니다.

### 5.1 코시안 유사도 기반, 검색
1. 기본값은 코사인 유사도인 similarity가 적용되어 있음
2. maximum marginal search result 적용

### 5.2 MMR 방식(maximum marginal search result) 
- MMR은 "최대 다양성 보장" 검색 방식
- 단순 유사도 기반이 아닌 유사성과 다양성을 동시에 고려
- 동작방식:
- 1. 가장 유사한 문서를 먼저 선택
- 2. 이미 선택된 문서들과 겹치지 않으며 질문들과 유사한 문서를 반복적으로 선택
- 3. 결과적으로 중복이 적고, 다양한 정보가 포함된 결과를 반환

In [ ]:
# query = "신분별 대출은 언제까지 가능해?"

# # 1.코사인유사도 기반 검색: similarity_score_threshold 이상인 결과만 반환
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"score_threshold": 0.8})
# search_result = retriever.get_relevant_documents(query)
# print(search_result)


In [12]:
# maximum marginal search result 적용
# 유사도 높은 K 개의 문서를 검색합니다.
query = "신분별 대출은 언제까지 가능해?"

mmr_retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
mmr_search_result = mmr_retriever.get_relevant_documents(query)
print(mmr_search_result)

/var/folders/w0/ll3gcfv51rj1zzw056sn6_d00000gn/T/ipykernel_73577/1987062988.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  mmr_search_result = mmr_retriever.get_relevant_documents(query)


[Document(id='26e06037-6b19-4287-b037-56ac1403926c', metadata={'source': 'https://hsel.hansung.ac.kr/intro_data.mir'}, page_content='③ 모든 도서의 대출은 예약자가 없는 경우 1회에 한하여 갱신이 가능하며, 대출 중인 도서 예약은 최대 2권까지 가능하다.\n④ 학부 및 대학원 졸업, 수료, 휴학생, 퇴직 교원 및 직원, 단기 교육생은 학술정보관 자료 열람 및 시설 이용을 허용한다.\n\n\n\n부칙\n\n(시행일) 본 규정은 2017년 7월 1일부터 시행한다.\n(시행일) 본 개정 규정은 2019년 11월 1일부터 시행한다.\n(시행일) 본 개정 규정은 2020년 10월 12일부터 시행한다.'), Document(id='75b6ab43-3dc4-46bd-8fe1-b4a3da428312', metadata={'source': 'book_data_RAG_location.csv', 'row': 2072}, page_content='서명: (잠든 사이 월급 버는) 미국 배당주 투자 :안정된 수익 내는 배당투자의 나침반\n저자: 소수몽키\n출판사: 베가북스\n출판년도: 2019\n청구기호: 327.856 ㅅ422ㅁ\n대출일자: 2022-04-27\n신분: 교직원\n소속(학과): 컴퓨터공학부\n학번: 교직원\n학년: \n대출횟수: 1\n보존서가 소장처: 사회과학자료실(4F)\n보존서가 칸: 20-A 6-a')]


# 6단계 프롬프트 생성
- 프롬프트 엔지니어링, 주어진 데이터를 토대로 우리가 원하는 결과가 나올 수 있도록 프롬프트를 수정하는것이다.

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
prompt
print(prompt)

In [15]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# 7단계: 언어모델 생성
# 모델 LLM을 생성합니다
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 문서를 포맷팅하는 함수(검색결과를 하나의 문단으로 합쳐줍니다.)
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [17]:
# 8단계: 체인 생성(Create Chain)
rag_chain = (
    {"context": mmr_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 8단계: 체인 실행
# 문서에 대한 질의를 입력하고 답변을 출력합니다.
question = "대출은 언제까지 가능한거야?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")



문서의 수: 26436
[HUMAN]
대출은 언제까지 가능한거야?

[AI]
도서 대출은 예약자가 없는 경우 1회에 한하여 갱신이 가능하며, 대출 중인 도서 예약은 최대 2권까지 가능하다. 대출 규정에 따라 대출 가능 기간은 정해져 있지 않으므로 도서관의 정책에 따라 다를 수 있습니다. 대출 가능 여부 및 기간은 해당 도서관의 규정을 확인해야 합니다.


In [18]:
# 결과값 테스트 책추천도 해주는지? Hallucination 유발 해볼게요
question = "책 추천해줘"
response = rag_chain.invoke(question)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")


[HUMAN]
책 추천해줘

[AI]
'미스터리 가이드북 :한 권으로 살펴보는 미스터리 장르의 모든 것'과 '나는 매일 책을 읽기로 했다'를 추천합니다.


In [19]:
# question 보완
question = "카리나 생일 언제야"
response = rag_chain.invoke(question)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

[HUMAN]
카리나 생일 언제야

[AI]
카리나 생일 언제야에 대한 정보는 제공된 문맥에서 찾을 수 없습니다.


In [20]:
# question 보완
question = "한성대학교는 뭐하는곳이야"
response = rag_chain.invoke(question)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

[HUMAN]
한성대학교는 뭐하는곳이야

[AI]
한성대학교는 대학과 지식을 다루는 곳이며, 컴퓨터공학부를 포함한 다양한 학과가 있습니다.
